In [ ]:
import os
os.environ["HDF5_PLUGIN_PATH"] = '/usr/lib/x86_64-linux-gnu/hdf5/serial/plugins'
import h5py

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm

import torch
from torch_geometric.data import  Data
import cv2
import random
import pickle
import time
import pandas as pd
import math
import glob
import csv
import sys

from event_utils.lib.data_formats.event_packagers import hdf5_packager
from event_utils.lib.data_formats.read_events import read_h5_events_dict

In [ ]:
def split_array_by_window(arr, t_len):
    # Sort the input array by timestamps if it's not already sorted


    # Find the minimum and maximum timestamps
    min_timestamp, max_timestamp = arr.min(), arr.max()

    # Calculate the number of windows required
    num_windows = int(np.ceil((max_timestamp - min_timestamp) / t_len))

    # Initialize a list to store the split arrays
    split_arrays = []
    split_indexes = []

    # Iterate through the windows and split the array
    for i in tqdm(range(num_windows)):
        window_start = min_timestamp + i * t_len
        window_end = min_timestamp + (i + 1) * t_len

        # Extract elements within the current window
        split_index = np.where((arr >= window_start) & (arr < window_end))[0]
        split_indexes.append(split_index)
        window_array = arr[split_index] - window_start

        if len(window_array) > 0:
            split_arrays.append(window_array)

    return split_arrays,split_indexes

In [ ]:
download_path = '../datasets/led'
classes = glob.glob(os.path.join(download_path , '*'))
hdf5_pathes = glob.glob(os.path.join(download_path , '*/*.hdf5'))

In [ ]:
evt_list = []
split_arrays_list = []
split_indexes_list = []
num_events_list = []
t_len = 74000 # in micro seconds
for hdf5_path in hdf5_pathes:
    print(hdf5_path)
    class_name = os.path.dirname(hdf5_path).split(os.path.sep)[-1]
    dir_path = os.path.dirname(hdf5_path).replace('full_video','segmented')
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        
    with h5py.File(hdf5_path, "r") as f:
        evts = f["CD"]["events"][()]
          
        timestamps = evts['t']
        split_arrays,split_indexes = split_array_by_window(timestamps, t_len)
        split_arrays_list.append(split_arrays)
        split_indexes_list.append(split_indexes)
        
        evts_windows = []
        for i in range(len(split_arrays)):
            idx = split_indexes[i]
            assert len(idx) == len(split_arrays[i])
            evts_window = evts[idx]
            evts_window['t'] = split_arrays[i]
            evts_windows.append(evts_window)
            
            file_name = f'{class_name}_{i:04d}.hdf5'
            hdf5_path = os.path.join(dir_path,file_name)
              
            packager = hdf5_packager(hdf5_path)
            packager.package_events(evts_window['x'],evts_window['y'],evts_window['t'],evts_window['p'])
            
            num_pos = np.sum(evts_window['p'] == 1)
            num_neg = np.sum(evts_window['p'] == 0)
            t0 = evts_window['t'][0]
            tk = evts_window['t'][-1]
            duration = tk - t0
            num_imgs = 0
            num_flow = 0
            sensor_size = [1280, 720]
            
            packager.add_metadata(num_pos, num_neg, duration, t0, tk, num_imgs, num_flow, sensor_size)
                     
            packager.events_file.close()
            
        evt_list.append(evts_windows)
        
        num_events = np.array([len(split_arrays[i]) for i in range(len(split_arrays))])
        num_events_list.append(num_events)
        

In [ ]:
i = 89
class_name = os.path.dirname(hdf5_path).split(os.path.sep)[-1]
dir_path = os.path.join(os.path.dirname(hdf5_path),class_name)
file_name = f'{class_name}_{i:04d}.hdf5'
file_path = os.path.join(dir_path,file_name)
print(file_path)   

In [ ]:

events = read_h5_events_dict('datasets_torch_geometric/fan1vs3/data/raw/all/speed_1/speed_1_0000.hdf5', read_frames=False)

In [ ]:
data_x = events['xs'].astype(np.float32)
data_y = events['ys'].astype(np.float32)
data_ts = events['ts'].astype(np.float32)    
data_p = events['ps'].astype(np.float32)

pos = np.array([data_x,data_y,data_ts])
pos = torch.from_numpy(pos)
pos = pos.transpose(0,1)
data_p = torch.from_numpy(data_p)
data = Data(x=data_p,pos=pos)

In [ ]:
data.pos

In [ ]:
# plot histograms of arrays in 'num_events_list' in one figure with defferent colors

for i in range(len(num_events_list)):
    plt.hist(num_events_list[i], bins=100, alpha=0.5, label=classes[i])